## Distinguishing Failed Experiments with Rubicon 
When logging with `rubicon_ml` , if an experiment fails, it will result in an empty experiment. In this example, we'll walk through how to handle these experiments. 

First lets simulate the problem. To do this we'll create an estimator that will fail on it's `fit()` by throwing a `RuntimeError`.

In [1]:
from sklearn.base import BaseEstimator

class BadEstimator(BaseEstimator):
    def __init__(self, n_neighbors=2):
        super().__init__()
        
        self.n_neighbors = n_neighbors
        
        self.knn = KNeighborsClassifier(n_neighbors=n_neighbors)
    def fit(self, X, y):
        self.knn.fit(X, y)
        self.knn.result="fit"
        try:
            raise RuntimeError("Bad Estimator")
        except RuntimeError:
            pass
        
    def score(self, X):
        knn_score = self.knn.score(X)
        return knn_score

Now let's attempt to create a `rubicon_ml.sklearn` pipeline with this failing estimator and attempt to `fit` the pipeline. 

In [2]:
from rubicon_ml.sklearn import make_pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer



from rubicon_ml import Rubicon

rubicon = Rubicon(
    persistence="memory",
)
project = rubicon.get_or_create_project(name="Failed Experiment")

X = [[1], [1], [1], [1]]
y = [1, 1, 1, 1]
rf=RandomForestClassifier(n_estimators=50)
pipe=make_pipeline(project, SimpleImputer(strategy="mean"),BadEstimator())
pipe.fit(X,y)
pipe.fit(X,y)
print(len(project.experiments()))
for exp in project.experiments():
    print(" parameters are :")
    for p in exp.parameters():
        print(p.__dict__)

2
 parameters are :
{'_config': <rubicon_ml.client.config.Config object at 0x169a86ec0>, '_domain': Parameter(name='simpleimputer__add_indicator', id='695c2c10-e630-43ab-896d-ee2aabead479', value=False, description=None, created_at=datetime.datetime(2022, 5, 6, 19, 40, 20, 569328))}
{'_config': <rubicon_ml.client.config.Config object at 0x169a86ec0>, '_domain': Parameter(name='simpleimputer__copy', id='e7fce379-01af-4610-ae2a-dd2b6f8b3f23', value=True, description=None, created_at=datetime.datetime(2022, 5, 6, 19, 40, 20, 569488))}
{'_config': <rubicon_ml.client.config.Config object at 0x169a86ec0>, '_domain': Parameter(name='simpleimputer__fill_value', id='79c24baa-6bd3-458e-91a0-ecf21e2f8137', value=None, description=None, created_at=datetime.datetime(2022, 5, 6, 19, 40, 20, 569584))}
{'_config': <rubicon_ml.client.config.Config object at 0x169a86ec0>, '_domain': Parameter(name='simpleimputer__missing_values', id='4e7668c5-8c68-415e-863d-8c9be891cee0', value=nan, description=None, cr

When an experiment is create we can check its contents for length. Here we can see nothing has been logged.

In [3]:
exp=project.experiments()[0]
print(exp.__dict__)
assert len(exp.artifacts())==0
assert len(exp.dataframes())==0
assert len(exp.metrics())==0
assert len(exp.features())==0

{'_config': <rubicon_ml.client.config.Config object at 0x169a86ec0>, '_domain': Experiment(project_name='Failed Experiment', id='309f708f-8ed3-4a11-bebd-65c5591f5fbe', name='RubiconPipeline experiment', description=None, model_name=None, branch_name=None, commit_hash=None, training_metadata=None, tags=[], created_at=datetime.datetime(2022, 5, 6, 19, 40, 20, 569147)), '_parent': <rubicon_ml.client.project.Project object at 0x169a86e30>, '_artifacts': [], '_dataframes': [], '_metrics': [], '_features': [], '_parameters': []}


Since we have observed an empty experiment, we need to identify it as a failed experiment. We can do this by leveraging the `tags` attribute for the experiment with `add_tags()`

In [4]:
exp.add_tags(["failed"])

Finally, we can no retrieve all our failed experiments by passing the `tags=["failed"]` to `project.experiments()`.

In [5]:
project.experiments(tags=["failed"])